In [1]:
import sys
import os

path = os.path.abspath('..')
if path not in sys.path:
    sys.path.insert(0, os.path.abspath('..'))
    
sys.path

['/home/justin/Github/aqbuildtools',
 '/home/justin/Github/aqbuildtools/examples',
 '/home/justin/anaconda3/envs/aqbt/lib/python38.zip',
 '/home/justin/anaconda3/envs/aqbt/lib/python3.8',
 '/home/justin/anaconda3/envs/aqbt/lib/python3.8/lib-dynload',
 '',
 '/home/justin/anaconda3/envs/aqbt/lib/python3.8/site-packages',
 '/home/justin/anaconda3/envs/aqbt/lib/python3.8/site-packages/IPython/extensions',
 '/home/justin/.ipython']

In [10]:
from aqbt import AquariumBuildTools
aqtools = AquariumBuildTools.from_toml('creds.secret.toml')
aqtools.sessions

aq = aqtools.sessions['local']['aquarium']

TridentLoginError: Could not contact 'http://0.0.0.0'. Login request returned no response

In [ ]:
aqtools.sessions

In [3]:
import pydent
from aqbt.contrib.uwbf import primers

primers.get_aq_primers_df(aq)

  0%|          | 0/36221 [00:00<?, ?it/s]

,id,name,anneal,overhang,sequence
0,1,IAA1-Nat-F,ATGGAAGTCACCAATGGGCTTAACCTTAAG,AAAAAGCAGGCTTCAAA,AAAAAGCAGGCTTCAAAATGGAAGTCACCAATGGGCTTAACCTTAAG
1,14,prKL1573,AGTTTATCATTATCAATACTCGCCATTTCAAAGAA,CCTTAACCAGATTCGAAAAGCGGC,CCTTAACCAGATTCGAAAAGCGGCAGTTTATCATTATCAATACTCG...
2,15,prKL744,ATCCACTAGTTCTAGAATCCGTCGAAACTAAGTT,TTTGTAG,TTTGTAGATCCACTAGTTCTAGAATCCGTCGAAACTAAGTT
3,35,prKL1927,AACACCCCTTGTATTACTGTTTATGTAAGCAGACA,,AACACCCCTTGTATTACTGTTTATGTAAGCAGACA
4,36,prKL1928,TTTTTCCCTCCTTACAGTTTCTGATTTGTG,TGAAAAGTTCTTCTCCTTTACGCAT,TGAAAAGTTCTTCTCCTTTACGCATTTTTTCCCTCCTTACAGTTTC...
...,...,...,...,...,...
9571,36172,PRSV_Fwd_SEQ6,GCCTGATGCGGTATTTTC,,GCCTGATGCGGTATTTTC
9572,36173,PRSV_Fwd_SEQ7,GTACAATCTGCTCTGATGCC,,GTACAATCTGCTCTGATGCC
9573,36174,PRSV_REV_SEQ8,CCGGGAAGCTAGAGTAAGTA,,CCGGGAAGCTAGAGTAAGTA
9574,36221,152 1st_SSM_F_petcF,,,


In [10]:
from pydent import AqSession

aq.Sample.pagination

<bound method QueryInterface.pagination of <pydent.interfaces.QueryInterface object at 0x7fe7810e1ee0>>

In [20]:
from tqdm.auto import tqdm

def pagination(query, page_size, model="Sample", pbar=True, pbar_desc=None, limit=None):
    
    if limit:
        last_id = aq.model_interface(model).first(1, query=query, opts={'offset': limit})[0].id
    else:
        last_id = aq.model_interface(model).last(1, query=query)[0].id

    opts = {}
    if limit:
        opts['limit'] = limit
    iterator = aq.model_interface(model).pagination(query=query, page_size=page_size, opts=opts)
    if pbar:
        pbar_iter = tqdm(iterator, total=last_id)
    
        if pbar_desc:
            pbar_iter.set_description(pbar_desc)
        last_max_id = -1
        for primers in pbar_iter:
 
            ids = [m.id for m in primers]
            max_id = max(ids)
            pbar_iter.update(max_id - last_max_id)
            last_max_id = max_id
            yield primers
    else:
        yield from iterator
        
def get_primers(sess, page_size=500, pbar=True, limit=None):
    all_primers = []
    for primers in pagination({'sample_type_id': sess.SampleType.find_by_name('Primer').id}, page_size=page_size, limit=limit, pbar=True, pbar_desc='collecting primer seqs'):
        sess.browser.get(primers, {'sample_type': 'field_types', 'field_values': 'field_type'})
        all_primers += primers
    return all_primers

def clean_seq(seq):
    return re.sub('\s', '', seq)

def _primer_add_seqs(p):
    try:
        p.properties
        overhang = clean_seq(p.properties['Overhang Sequence'] or '')
        anneal = clean_seq(p.properties['Anneal Sequence'] or '')
        
        sequence = overhang + anneal
        p.overhang = overhang
        p.anneal = anneal
        p.sequence = sequence
    except JSONDecodeError as e:
        pass
    
def _primer_rows(primers):
    for p in primers:
        _primer_add_seqs(p)
        
    primer_rows = []
    for p in primers:
        if hasattr(p, 'anneal'):
            row = {
                'id': p.id,
                'name': p.name,
                'anneal': p.anneal,
                'overhang': p.overhang,
                'sequence': p.sequence
            }
            primer_rows.append(row)
    return primer_rows

def primer_df(primers):
    primer_rows = _primer_rows(primers)
    df = pd.DataFrame(primer_rows)
    return df
    

In [ ]:
with aq.with_cache(timeout=50) as sess:
    primers = get_primers(sess)

In [126]:
primer_df = primer_df(primers)
primer_df.head()

,id,name,anneal,overhang,sequence
0,1,IAA1-Nat-F,ATGGAAGTCACCAATGGGCTTAACCTTAAG,AAAAAGCAGGCTTCAAA,AAAAAGCAGGCTTCAAAATGGAAGTCACCAATGGGCTTAACCTTAAG
1,14,prKL1573,AGTTTATCATTATCAATACTCGCCATTTCAAAGAA,CCTTAACCAGATTCGAAAAGCGGC,CCTTAACCAGATTCGAAAAGCGGCAGTTTATCATTATCAATACTCG...
2,15,prKL744,ATCCACTAGTTCTAGAATCCGTCGAAACTAAGTT,TTTGTAG,TTTGTAGATCCACTAGTTCTAGAATCCGTCGAAACTAAGTT
3,35,prKL1927,AACACCCCTTGTATTACTGTTTATGTAAGCAGACA,,AACACCCCTTGTATTACTGTTTATGTAAGCAGACA
4,36,prKL1928,TTTTTCCCTCCTTACAGTTTCTGATTTGTG,TGAAAAGTTCTTCTCCTTTACGCAT,TGAAAAGTTCTTCTCCTTTACGCATTTTTTCCCTCCTTACAGTTTC...


In [128]:
def is_left_end_terminal(df):
    template_len = len(df.meta['template'])
    return (df['strand'] == 1) & (df['start'] == 0)

def is_right_end_terminal(df):
    template_len = len(df.meta['template'])
    return (df['strand'] == -1) & (df['start'] == template_len - 1)

def anneal_df(template, primers, names, n_bases=12, min_tm=None, max_overhang=None, strand=None, region=None):
    meta = {
        'template': template,
        'n_bases': n_bases,
        'min_tm': min_tm,
        'max_overhang': max_overhang,
        'strand': strand,
        'region': region
    }
    primer_list = list(zip(primers, names))
    fwd, rev = primer3plus.utils.anneal_iter(template, primer_list, n_bases=n_bases)
    
    filtered_fwd, filtered_rev = [], []
    
    rows = []
    for p in fwd + rev:
        seq = p['anneal'][-60:]
        tm = round(primer3.calcTm(seq), 2)
        p['tm'] = tm
        rows.append(p)
        
    df = pd.DataFrame(rows)
    if strand is not None:
        assert strand in [1, -1]
        df = df[df['strand'] == strand]
    if min_tm is not None:
        df = df[df['tm'] >= min_tm]
        
    if region is not None:
        sel_a = df['top_strand_slice'].apply(lambda x: x[0] >= region[0])
        df = df[sel_a]
        sel_b = df['top_strand_slice'].apply(lambda x: x[1] < region[1])
        df = df[sel_b]
    if max_overhang is not None:
        sel = df['overhang'].apply(lambda x: len(x)) <= max_overhang
        df = df[sel]
    df.meta = meta
    df['left_term'] = is_left_end_terminal(df)
    df['right_term'] = is_right_end_terminal(df)
    return df

binding_df = anneal_df(template, primer_df.sequence, primer_df.name, min_tm=45)
binding_df.tail()

<ipython-input-128-ae2fd7d2a56c>:45: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.meta = meta


,name,anneal,overhang,primer,start,length,top_strand_slice,strand,tm,left_term,right_term
86,Venus Reverse,catcaccttcaccttcaccgg,,catcaccttcaccttcaccgg,105,21,"(85, 106)",-1,55.13,False,False
88,yEVENUS_seq_r,TCACCGGAGACAGAAAATTTGTG,,TCACCGGAGACAGAAAATTTGTG,91,23,"(69, 92)",-1,53.59,False,False
89,eGFP_g93t_stop_rev,accggagacagaaaatttgtgacc,ggtcaatttaccgtaagtagcatcaccttcacctta,ggtcaatttaccgtaagtagcatcaccttcaccttaaccggagaca...,89,24,"(66, 90)",-1,55.59,False,False
90,yVenus_RT_probe_1,ACAGAAAATTTGTGACCATT,,ACAGAAAATTTGTGACCATT,82,20,"(63, 83)",-1,45.59,False,False
91,yeGFP_reverse,CTAATTCAACCAAAATTGGGACAACACC,,CTAATTCAACCAAAATTGGGACAACACC,51,28,"(24, 52)",-1,55.82,False,False


In [121]:
def is_left_end_terminal(df):
    template_len = len(df.meta['template'])
    return (df['strand'] == 1) & (df['start'] == 0)

def is_right_end_terminal(df):
    template_len = len(df.meta['template'])
    return (df['strand'] == -1) & (df['start'] == template_len - 1)

is_left_end_terminal(binding_df)

5      True
6      True
7      True
8      True
9      True
      ...  
86    False
88    False
89    False
90    False
91    False
Length: 76, dtype: bool

In [124]:
df = anneal_df("gaattattcactggtgttgtcccaattttggttgaattagatggtgatgttaatggtcacaaattttctgtctccggtgaaggtgaagg", [rc('aattttctgtctccggtgaaggtgaagg')], [''])

is_right_end_terminal(df)

<ipython-input-113-6d665c8f9c61>:37: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.meta = meta


0    True
dtype: bool

In [115]:
len('gaattattcactggtgttgtcccaattttggttgaattagatggtgatgttaatggtcacaaattttctgtctccggtgaaggtgaagg')

89

In [169]:
from typing import *

def _design_primers(
    template: str,
    region: Tuple[int, int],
    lseq: Union[None, str],
    rseq: Union[None, str],
    left_overhang: Union[None, str] = None,
    right_overhang: Union[None, str] = None,
) -> Tuple[Dict[int, dict], Dict[str, Any]]:
    """Design primers flanking the specified.

    :class:`Region.<dasi.utils.Region>`. If the region is cyclic and spans the
    origin, this method will handle the appropriate manipulations to design
    primers around the origin and restore the locations of the resulting primer
    pairs.

    :param template: the template string to design primers
    :param region: region specified to design primers around. Regions are exclusive at
                    their end points (`.b` parameter)
    :param lseq: optionally provided left sequence
    :param rseq: optionally provided right sequence
    :param left_overhang: optionally provided left overhang sequence of the primer
    :param right_overhang: optionally provided right overhang sequence of the primer
    :return: tuple of pairs and the 'explain' dictionary.
    """
    design = primer3plus.new()
    design.settings.as_cloning_task()
    
#     if region.direction == -1:
#         region = region.flip()
#         template = rc(template)

    if lseq and left_overhang:
        raise DasiSequenceDesignException
    if rseq and right_overhang:
        raise DasiSequenceDesignException

    spans_origin = False
    if spans_origin:
        adjusted_template = region.get_slice(template) + region.invert()[0].get_slice(
            template
        )
        design.settings.template(adjusted_template)
        design.settings.included((0, ))
        index = list(region) + list(region.invert()[0])
    else:
        design.settings.template(template)
        design.settings.included((region[0], region[1]))
        index = None
    
    if lseq:
        design.settings.left_sequence(lseq)
    if rseq:
        design.settings.right_sequence(rseq)

    if left_overhang is None:
        left_overhang = ""
    if right_overhang is None:
        right_overhang = ""

    design.settings.product_size((region[1], region[1]))
    design.settings.left_overhang(left_overhang)
    design.settings.right_overhang(right_overhang)
    design.PRIMER_PICK_ANYWAY = False
    design.PRIMER_MIN_ANNEAL_CHECK = 15
    design.settings.use_overhangs()
    design.settings.long_ok()

    design.logger.set_level("INFO")

    # TODO: remove debugging code
    try:
        pairs, explain = design.run_and_optimize(
            3, pick_anyway=True
        )
    except Exception as e:
        import json

        raise e
    if index is not None:
        for pair in pairs.values():
            loc = pair["LEFT"]["location"]
            pair["LEFT"]["location"] = (index[loc[0]], loc[1])

            loc = pair["RIGHT"]["location"]
            pair["RIGHT"]["location"] = (index[loc[0]], loc[1])
    return pairs, explain

template = 'tctaaaggtgaagaattattcactggtgttgtcccaattttggttgaattagatggtgatgttaatggtcacaaattttctgtctccggtgaaggtgaaggtgatgctacttacggtaaattgaccttaaaatttatttgtactactggtaaattgccagttccatggccaaccttagtcactactttcggttatggtgttcaatgttttgcgagatacccagatcatatgaaacaacatgactttttcaagtctgccatgccagaaggttatgttcaagaaagaactatttttttcaaagatgacggtaactacaagaccagagctgaagtcaagtttgaaggtgataccttagttaatagaatcgaattaaaaggtattgattttaaagaagatggtaacattttaggtcacaaattggaatacaactataactctcacaatgtttacatcatggctgacaaacaaaagaatggtatcaaagttaacttcaaaattagacacaacattgaagatggttctgttcaattagctgaccattatcaacaaaatactccaattggtgatggtccagtcttgttaccagacaaccattacttatccactcaatctgccttatccaaagatccaaacgaaaagagagaccacatggtcttgttagaatttgttactgctgctggtattacccatggtatggatgaattgtacaaa'

_design_primers(template, (0, 200), None, None)

INFO - Design(id=140148930136000) - 2021-02-24 09:49:29,144 - Updated: {'PRIMER_MAX_SIZE': 28, 'PRIMER_MIN_SIZE': 17, 'PRIMER_MAX_TM': 64.0, 'PRIMER_MIN_TM': 56.0, 'PRIMER_MAX_HAIRPIN_TH': 48.0}
INFO - Design(id=140148930136000) - 2021-02-24 09:49:29,147 - Updated: {'PRIMER_MAX_SIZE': 29, 'PRIMER_MIN_SIZE': 16, 'PRIMER_MAX_TM': 65.0, 'PRIMER_MIN_TM': 55.0, 'PRIMER_MAX_HAIRPIN_TH': 49.0}
INFO - Design(id=140148930136000) - 2021-02-24 09:49:29,152 - Updated: {'PRIMER_MAX_SIZE': 30, 'PRIMER_MAX_TM': 66.0, 'PRIMER_MIN_TM': 54.0, 'PRIMER_MAX_HAIRPIN_TH': 50.0}


({0: {'PAIR': {'PENALTY': 12.636461644997382,
    'COMPL_ANY_TH': 0.0,
    'COMPL_END_TH': 0.9191653717943495,
    'PRODUCT_SIZE': 200},
   'LEFT': {'PENALTY': 8.35375885684357,
    'PROBLEMS': ' Hairpin stability too high;',
    'SEQUENCE': 'tctaaaggtgaagaattattcactggtg',
    'location': (0, 28),
    'TM': 59.64624114315643,
    'GC_PERCENT': 35.714285714285715,
    'SELF_ANY_TH': 33.16317392155554,
    'SELF_END_TH': 22.71514578820046,
    'HAIRPIN_TH': 51.56381098775216,
    'END_STABILITY': 4.17,
    'OVERHANG': ''},
   'RIGHT': {'PENALTY': 4.282702788153813,
    'SEQUENCE': 'acaccataaccgaaagtagtgact',
    'location': (199, 24),
    'TM': 59.71729721184619,
    'GC_PERCENT': 41.666666666666664,
    'SELF_ANY_TH': 0.0,
    'SELF_END_TH': 0.0,
    'HAIRPIN_TH': 37.594332539309505,
    'END_STABILITY': 3.41,
    'OVERHANG': ''}},
  1: {'PAIR': {'PENALTY': 12.665878250085882,
    'COMPL_ANY_TH': 0.0,
    'COMPL_END_TH': 0.0,
    'PRODUCT_SIZE': 200},
   'LEFT': {'PENALTY': 8.3831754619

In [167]:
rc('acaccataaccgaaagtagtgact')

'agtcactactttcggttatggtgt'

In [166]:
design.settings.primer_num_return = 1